## Modelo de classificação Naive Bayes


In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from sklearn.preprocessing import OneHotEncoder
import numpy as np



### Limpeza e tratamento do dataframe

In [7]:
df = pd.read_csv('/Users/manu/Documents/Projetos Ciência de Dados/Naive Bayes|Arvores de decisao/Telco_Customer_Churn_dataset.csv')
df.shape
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [8]:
#Verificação de informações básicas do DataFrame
print(df.info())

print(df.isnull().sum())

print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [9]:
#Tratamento do tipo de dados
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].astype(float)

#One Hot Encoding
df_encoded = pd.get_dummies(df, columns=['gender', 'Partner', 'Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies', 'Contract', 'PaymentMethod', 'PaperlessBilling'], drop_first=True)

# Exibir as primeiras linhas do DataFrame codificado
df_encoded = df_encoded.dropna()
df_encoded.head()

,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,...,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,PaperlessBilling_Yes
0,7590-VHVEG,0,1,29.85,29.85,No,False,True,False,False,...,False,False,False,False,False,False,False,True,False,True
1,5575-GNVDE,0,34,56.95,1889.50,No,True,False,False,True,...,False,False,False,False,True,False,False,False,True,False
2,3668-QPYBK,0,2,53.85,108.15,Yes,True,False,False,True,...,False,False,False,False,False,False,False,False,True,True
3,7795-CFOCW,0,45,42.30,1840.75,No,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False
4,9237-HQITU,0,2,70.70,151.65,Yes,False,False,False,True,...,False,False,False,False,False,False,False,True,False,True


In [10]:
#Definição das variáveis previsoras e target
X = df_encoded.drop(columns=['customerID', 'Churn'])
y = df_encoded['Churn']

print(X.shape, y.shape)

(7032, 30) (7032,)


In [11]:
# Formato de matriz
previsores = X.values
classe = y.values

In [12]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,
                                                                  classe,
                                                                  test_size = 0.3,
                                                                  random_state = 0)

### GaussianNB
O GaussianNB é uma variante do algoritmo Naive Bayes que assume que as features (variáveis independentes) seguem uma distribuição normal (Gaussiana).
* Indicado para dados contínuos, onde as features podem assumir qualquer valor real.

In [13]:
# Criação e treinamento do modelo (geração da tabela de probabilidades)
naive_bayes = GaussianNB()
naive_bayes.fit(X_treinamento, y_treinamento)

GaussianNB()

In [14]:
# Previsões utilizando os registros de teste
previsoes = naive_bayes.predict(X_teste)
previsoes

array(['No', 'No', 'No', ..., 'Yes', 'No', 'No'], dtype='<U3')

In [15]:
#geração da matriz de confusão e cálculo da taxa de acerto e erro
confusao = confusion_matrix(y_teste, previsoes)
confusao

array([[942, 613],
       [ 73, 482]])

#### Acurácia do modelo

In [17]:
taxa_acerto = accuracy_score(y_teste, previsoes)
taxa_erro = 1 - taxa_acerto
taxa_acerto

print('\nClassification Report:\n', classification_report(y_teste, previsoes))


Classification Report:
               precision    recall  f1-score   support

          No       0.93      0.61      0.73      1555
         Yes       0.44      0.87      0.58       555

    accuracy                           0.67      2110
   macro avg       0.68      0.74      0.66      2110
weighted avg       0.80      0.67      0.69      2110



### BernoulliNB
O BernoulliNB é uma variante do Naive Bayes que assume que as features são binárias (0 ou 1), seguindo uma distribuição de Bernoulli.
* É adequado para dados binários ou que podem ser transformados em binários.

In [23]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.preprocessing import Binarizer


# Binarização dos dados
binarizer = Binarizer(threshold=0.0)
X_treinamento_binario = binarizer.fit_transform(X_treinamento)
X_teste_binario = binarizer.transform(X_teste)

# Criação e treinamento do modelo BernoulliNB
naive_bayes_B = BernoulliNB()
naive_bayes_B.fit(X_treinamento_binario, y_treinamento)

# Previsões no conjunto de teste
previsoes_Bernoulli = naive_bayes_B.predict(X_teste_binario)

# Acurácia
acuracia_Bernoulli = accuracy_score(y_teste, previsoes_Bernoulli)
print(f"Acurácia do modelo: {acuracia_Bernoulli:.2f}")
print('\nClassification Report:\n', classification_report(y_teste, previsoes_Bernoulli))
# Matriz de confusão
matriz_confusao_Bernoulli = confusion_matrix(y_teste, previsoes_Bernoulli)
print("Matriz de Confusão:")
print(matriz_confusao_Bernoulli)

Acurácia do modelo: 0.72

Classification Report:
               precision    recall  f1-score   support

          No       0.90      0.70      0.79      1555
         Yes       0.48      0.79      0.60       555

    accuracy                           0.72      2110
   macro avg       0.69      0.74      0.69      2110
weighted avg       0.79      0.72      0.74      2110

Matriz de Confusão:
[[1087  468]
 [ 119  436]]


##### ***Observação*** : Devido à mistura de variáveis categóricas, discretas e contínuas na base de dados utilizada, o GaussianNB e o BernoulliNB não são as melhores escolhas. No entanto, esta base de dados foi utilizada para fins de estudo dos modelos de Naive Bayes
